### Imports and data loading

In [219]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import wkt
import folium

In [ ]:
data_root = "../data"

In [12]:
geo_data = gpd.read_file(f'{data_root}/reglementation-des-peches-cartographiee.gpkg', encoding='utf-8')

In [13]:
data = pd.read_csv(f'{data_root}/reglementation-des-peches-cartographiee.csv')

In [11]:
data.wkt = gpd.GeoSeries.from_wkt(data.wkt)

In [223]:
data_csv_geo = gpd.GeoDataFrame(data, geometry='wkt')


### Data exploration

In [224]:
data_csv_geo.sample(5)

,type_de_reglementation,thematique,zone,reglementations,wkt
149,Reg. MED,Occitanie - Dragues,Zone d'autorisation : Sète-Mole,Arrêté Ministériel du 13/05/2014,"MULTIPOLYGON (((3.62115 43.29380, 3.56624 43.3..."
396,Reg. MEMN,Normandie - Crustacés - Casiers et Filets,Interdiction de casiers pièges sur la bande co...,Arrêté Préfectoral 174/2019 - Délib 2019/28-CR...,GEOMETRYCOLLECTION (MULTIPOLYGON (((-1.58308 4...
439,Reg. MEMN,Ouest Cotentin - Casiers - Bulots,Grandville : zone d'entrepôt des casiers,Arrêté Préfectoral 133/2017 / DELIB 2017/29-BU...,"MULTIPOLYGON (((-1.84742 48.85995, -1.57998 48..."
406,Reg. MEMN,Normandie - Moules,Gisement de L'épée et le Vilain,"Arrêté Préfectoral 56/2018 / MEMN, Arrêté Préf...","MULTIPOLYGON (((-0.64028 49.56098, -0.63648 49..."
529,Reg. NAMO,Bretagne - Ormeaux,Zone 4: Hors Molène et et Ouessant (4A et 4C) ...,Arrêté Préfectoral R3-2022-09-20-00005 - Délib...,"MULTIPOLYGON (((-4.55308 48.16801, -5.35146 48..."


In [225]:
geo_data.sample(5)

,type_de_reglementation,thematique,zone,reglementations,geometry
265,Reg. NAMO,Pays de la Loire - Coquillage - Drague,Banc de Mindin - dispositions spécifiques à l'...,Arrêté Préfectoral 31/2015 - délib 8B/2015 / N...,"MULTIPOLYGON (((-2.21833 47.25067, -2.18800 47..."
302,Reg. NAMO,Saint-Brieuc - Chalut - Maquereau,Bande 3-12 MN,Arrêté Préfectoral R53-2019-10-29-002 du 29/10...,"MULTIPOLYGON (((-3.68988 48.81066, -3.71048 48..."
98,Reg. NAMO,Civelle,Secteur dit de l’Aiguillon-sur-Mer (le Lay),Arrêté Préfectoral 96/DRAM/2077 / NAMO,"MULTIPOLYGON (((-1.37061 46.39532, -1.37061 46..."
154,Reg. NAMO,Bretagne - Nasses,Secteur 8 - zone soumise à licence spéciale,Arrêté Préfectoral R53-2022-12-08-00001 - déli...,"MULTIPOLYGON (((-4.08263 47.46665, -4.08263 47..."
88,Reg. NAMO,Civelle,Secteur dit de Noirmoutier (etier des champs),Arrêté Préfectoral 96/DRAM/2077 / NAMO,"MULTIPOLYGON (((-1.98285 46.96480, -1.98285 46..."


### Check for completude of the data

In [226]:
data_csv_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   type_de_reglementation  882 non-null    object  
 1   thematique              882 non-null    object  
 2   zone                    879 non-null    object  
 3   reglementations         879 non-null    object  
 4   wkt                     882 non-null    geometry
dtypes: geometry(1), object(4)
memory usage: 34.6+ KB


In [227]:
geo_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   type_de_reglementation  331 non-null    object  
 1   thematique              331 non-null    object  
 2   zone                    331 non-null    object  
 3   reglementations         331 non-null    object  
 4   geometry                331 non-null    geometry
dtypes: geometry(1), object(4)
memory usage: 13.1+ KB


In [228]:
data_csv_geo[data_csv_geo.zone.isna()]

,type_de_reglementation,thematique,zone,reglementations,wkt
813,Reg. ORGP,OPANO/NAFO - Crevette nordique,NaN,NaN,"MULTIPOLYGON (((-45.66667 47.50000, -45.00000 ..."
814,Reg. ORGP,OPANO/NAFO - Pêche de fond,NaN,NaN,"MULTIPOLYGON (((-45.00833 50.00833, -47.00833 ..."
815,Reg. ORGP,OPANO/NAFO - Crevette nordique,NaN,NaN,"MULTIPOLYGON (((-42.00000 49.25000, -42.00000 ..."


We have only 3 null values on **"zone"** and **"reglementations"** that corresponds to the same observations. They share the **same type of reglementation** and the **same region of the world**.

### Lets' see the number of unique values for each category

In [229]:
data_csv_geo.describe(include="all")

,type_de_reglementation,thematique,zone,reglementations,wkt
count,882,882,879,879,882
unique,11,228,858,295,836
top,Reg. NAMO,Civelle,Zone d'interdiction de pêche,R(UE) 2019/1241,"MULTIPOLYGON (((3.54670758 43.3760237, 3.54705..."
freq,331,26,4,55,10


In [230]:
data_csv_geo.type_de_reglementation.value_counts()

Reg. NAMO          331
Reg. MED           166
Reg. MEMN          152
Reg. Outre-mer      90
R(UE) 2019/1241     76
Reg. SA             49
R(UE) 1380/2013      8
Reg. ORGP            3
R(UE) 2022/1614      3
R(CE) 494/2002       2
Reg. CCAMLR          2
Name: type_de_reglementation, dtype: int64

In [231]:
geo_data.type_de_reglementation.value_counts()

Reg. NAMO    331
Name: type_de_reglementation, dtype: int64

In [232]:
data_csv_geo.reglementations.value_counts()

R(UE) 2019/1241                                                                                                                     55
Arrêté Préfectoral 96/DRAM/2077 / NAMO                                                                                              26
Arrêté Préfectoral 412/2008 / MED                                                                                                   23
Arrêté Préfectoral R53-2020-05-19-001 - délib 2020-006 / NAMO, Arrêté Préfectoral R53-2019-09-12-009 / Délib 2019-023 / NAMO CSJ    22
R(UE) 2019/1241 Modifié                                                                                                             21
                                                                                                                                    ..
Arrêté Préfectoral du 30/12/2014                                                                                                     1
Arrêté 73 du 04/07/1979 / NAMO, Arrêté Préfectoral R53-

In [233]:
data_csv_geo.zone.value_counts()

Zone d'interdiction de pêche       4
Zone réglementée 2                 3
Zone réglementée 1                 3
Baie de Bourgneuf                  3
Zone autorisée - Seine-Maritime    2
                                  ..
Coques Vilaine Dragues             1
Box merlu - Irlande                1
Box merlu - Golfe de Gascogne      1
Filet maillant et trémail          1
Zone E : réservée arts dormants    1
Name: zone, Length: 858, dtype: int64

In [234]:
data_csv_geo[data_csv_geo.zone == "Zone réglementée 2"]

,type_de_reglementation,thematique,zone,reglementations,wkt
378,Reg. MEMN,Ouest Cotentin - Dragues - Coquille St Jacques,Zone réglementée 2,Arrêté Préfectoral 147/2022 - délib 2022/E-CSJ...,"MULTIPOLYGON (((-2.08333 49.35424, -2.08333 49..."
604,Reg. Outre-mer,Saint Pierre et Miquelon - Concombre de mer,Zone réglementée 2,Arrêté Ministériel du 31/05/2022,"MULTIPOLYGON (((-56.79885 46.52194, -56.48167 ..."
607,Reg. Outre-mer,Saint Pierre et Miquelon - Crabe des neiges,Zone réglementée 2,Arrêté Ministériel du 01/04/2022 / Saint-Pierr...,"MULTIPOLYGON (((-56.15171 46.00005, -56.15171 ..."


In [235]:
data_csv_geo[data_csv_geo.zone == "Coques Vilaine Dragues"]

,type_de_reglementation,thematique,zone,reglementations,wkt
292,Reg. NAMO,Morbihan - bivalves,Coques Vilaine Dragues,Arrêté Préfectoral 2016-13780 - délib 2016-056...,"MULTIPOLYGON (((-2.51020 47.49575, -2.54363 47..."


<br>

First, we see that the data in the geopackage format only contains partial data, from only 1 region (NAMO). We see further down in this notebook that it seems to concern only the Brittanic littoral.

According to what we see, it seems that only the "zone" and the "wkt" columns could be interesting for our use case.

But there are 858 unique zones from 882 lines in the dataset. We should explore further. A first thing to test is maybe to have zones that contains the word "Interdiction" and separate them from other reglementations ("Temporary authorized fishing" for instance).

### Extraction of zones with a label indicating an "interdiction"

In [236]:
zones_interdites = []
for zone in data_csv_geo.zone.unique():
    zone = str(zone)
    if zone.find("interdi") != -1:
        zones_interdites.append(zone)

In [237]:
len(zones_interdites)

93

In [238]:
zones_interdites

['Banc de Maërl - Zone interdite à la pêche',
 'Banc de Maërl - interdiction de pêche',
 'Herbiers de zostères "Baie de Morlaix" - interdiction',
 "Kerfinel - zone d'interdiction temporaire",
 'Zone de cohabitation 2 - interdiction pêche',
 'Zone de cohabitation 1 - pêche interdite',
 'Eaux territoriales Bretagne - Pêche interdite',
 'Zone A - interdiction chalutage',
 'Zone B - interdiction chalutage',
 'Nord Groix - Zone interdite',
 'Sud Groix - Zone interdite',
 'Belle-Île - interdiction temporaire',
 'Quiberon - Zone interdite temporaire',
 "Zone d'interdiction : Ile-verte",
 "Zone d'interdiction : Epine-Est",
 "Zone d'interdiction : Agde",
 "Zone d'interdiction : Palavas",
 "Zone d'interdiction : Gruissan",
 "Zone d'interdiction : Fleury D'Aude",
 "Hérault et Gard - Pêche sous-marine interdite - Grau d'Adge",
 'Hérault et Gard - Pêche sous-marine interdite - Etang de Thau',
 "Zone d'interdiction 1 : Marseillan-Sète",
 "Zone d'interdiction 2 : Marseillan-Sète",
 "Zone d'interdicti

<br>

In [262]:
data_csv_geo.crs = "epsg:4326"
data_csv_geo.to_crs = {'init' :'epsg:4326'}

map_test = folium.Map(location=[48, -3.5], tiles="cartodbpositron", zoom_start=5)

<br>

#### Choose the number of samples to display on the map for performance matters

In [265]:
n_samples_for_viz = 100

<br>

In [266]:
data_csv_geo_viz = data_csv_geo.sample(n_samples_for_viz)

In [267]:
style1 = {'fillColor': '#0000BBFF', 'color': '#0000FF'}
style2 = {'fillColor': '#BB0000FF', 'lineColor': '#FF0000'}

In [268]:
folium.GeoJson(data=data_csv_geo_viz[~data_csv_geo_viz.zone.isin(zones_interdites)]["wkt"], style_function= lambda x: style1).add_to(map_test)

In [269]:
folium.GeoJson(data=data_csv_geo_viz[data_csv_geo_viz.zone.isin(zones_interdites)]["wkt"], style_function= lambda x: style2).add_to(map_test)

<br>

#### VISUALIZE ON MAP

In [ ]:
#RUN THIS TO DISPLAY MAP
map_test